# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Weather_Data = pd.read_csv("../WeatherPy/cities.csv")
Weather_Data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Salò,45.61,10.53,73.40,53,40,5.82,IT,1600866735
1,Hobart,-42.88,147.33,46.40,70,40,10.29,AU,1600866439
2,Parrita,9.52,-84.32,68.00,77,99,8.05,CR,1600866735
3,Yarmouth,41.71,-70.23,62.01,54,1,8.05,US,1600866735
4,Benguela,-12.58,13.41,77.58,65,23,10.27,AO,1600866735
...,...,...,...,...,...,...,...,...,...
549,Pacific Grove,36.62,-121.92,57.00,68,1,3.36,US,1600866955
550,Horizontina,-27.63,-54.31,72.00,68,0,3.00,BR,1600866792
551,Yen Bai,21.70,104.87,78.39,89,100,2.73,VN,1600866956
552,Gobabis,-22.45,18.97,93.04,11,82,14.12,NaN,1600866956


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = Weather_Data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = Weather_Data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
Narrowed_City_df = Weather_Data.loc[(Weather_Data["Wind Speed"] <= 10) & (Weather_Data["Cloudiness"] == 0) & \
                                   (Weather_Data["Max Temp"] >= 70) & (Weather_Data["Max Temp"] <= 80)].dropna()

Narrowed_City_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,East London,-33.02,27.91,73.40,56,0,8.05,ZA,1600866567
10,Celestún,20.87,-90.40,79.50,79,0,5.19,MX,1600866736
44,Port Elizabeth,-33.92,25.57,73.40,35,0,5.82,ZA,1600866572
53,Hermanus,-34.42,19.23,71.01,62,0,1.01,ZA,1600866573
76,Okny,47.54,29.46,77.88,18,0,9.26,UA,1600866612
137,Yangi Marg`ilon,40.43,71.72,71.60,33,0,6.93,UZ,1600866781
157,Kumul,42.80,93.45,72.54,24,0,6.98,CN,1600866795
162,Guerrero Negro,27.98,-114.06,72.63,73,0,8.16,MX,1600866520
182,Mount Isa,-20.73,139.50,77.00,25,0,9.17,AU,1600866645
203,Gilgit,35.92,74.31,79.74,15,0,5.08,PK,1600866815


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = Narrowed_City_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,East London,ZA,-33.02,27.91,
10,Celestún,MX,20.87,-90.40,
44,Port Elizabeth,ZA,-33.92,25.57,
53,Hermanus,ZA,-34.42,19.23,
76,Okny,UA,47.54,29.46,
137,Yangi Marg`ilon,UZ,40.43,71.72,
157,Kumul,CN,42.80,93.45,
162,Guerrero Negro,MX,27.98,-114.06,
182,Mount Isa,AU,-20.73,139.50,
203,Gilgit,PK,35.92,74.31,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 6: East London.
Closest hotel in East London is Premier Hotels & Resorts.
------------
Retrieving Results for Index 10: Celestún.
Closest hotel in Celestún is Castillito Kin-Nah.
------------
Retrieving Results for Index 44: Port Elizabeth.
Closest hotel in Port Elizabeth is Lemon Tree Lane Guest House.
------------
Retrieving Results for Index 53: Hermanus.
Closest hotel in Hermanus is One Marine Drive Boutique Hotel.
------------
Retrieving Results for Index 76: Okny.
Missing field/result... skipping.
------------
Retrieving Results for Index 137: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is ATLAS HOTEL.
------------
Retrieving Results for Index 157: Kumul.
Closest hotel in Kumul is 哈密环球大酒店.
------------
Retrieving Results for Index 162: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 182: Mount Isa.
Closest hotel in Mount Isa is ibis Styles Mt Isa Verona.
------------
Retrieving Re

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,East London,ZA,-33.02,27.91,Premier Hotels & Resorts
10,Celestún,MX,20.87,-90.40,Castillito Kin-Nah
44,Port Elizabeth,ZA,-33.92,25.57,Lemon Tree Lane Guest House
53,Hermanus,ZA,-34.42,19.23,One Marine Drive Boutique Hotel
76,Okny,UA,47.54,29.46,
137,Yangi Marg`ilon,UZ,40.43,71.72,ATLAS HOTEL
157,Kumul,CN,42.80,93.45,哈密环球大酒店
162,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
182,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
203,Gilgit,PK,35.92,74.31,Gilgit Serena Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))